# Modelos: LLM

### Papers

* [Chat-REC: Towards Interactive and Explainable
LLMs-Augmented Recommender System](https://github.com/adrianmarino/thesis-paper/blob/master/docs/ideas/2303.14524.pdf)
* [Large Language Models as Zero-Shot Conversational
Recommenders](https://github.com/adrianmarino/thesis-paper/blob/master/docs/ideas/3583780.3614949.pdf)
* [Large Language Models are Competitive Near Cold-start
Recommenders for Language- and Item-based Preferences](https://github.com/adrianmarino/thesis-paper/blob/master/docs/ideas/3604915.3608845.pdf)

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
BASE_PATH             = '../..'
LIB_PATH              = f'{BASE_PATH}/lib'
DATASET_PATH          = f'{BASE_PATH}/datasets'

In [6]:
import sys
sys.path.append(LIB_PATH)

import numpy as np
import pandas as pd

import util as ut

from rest.ollama import OllamaApiClient


from model.llm import MovieRecommenderOutputParser

## Setup

In [7]:
# !sudo systemctl enable ollama
# !sudo systemctl restart ollama

## Create model

Model definition:

In [8]:
! cat ./llms/Modelfile.movie_recommender

FROM llama2

# set the temperature to 1 [higher is more creative, lower is more coherent]
PARAMETER temperature 0.5

# set the system message
SYSTEM """
"""

In [9]:
# !sudo ollama create default -f ./Modelfile

## Promps

In [10]:
def promp_build_fn(**kargs):
    user_profile = kargs['user_profile'] if 'user_profile' in kargs else 'No existe informacion de su perfil'
    
    user_history = kargs['user_history'] if 'user_history' in kargs else 'Aun no se registraron peliculas vista'

    candidates = kargs['candidates'] if 'candidates' in kargs else 'Aun no se dispone de peliculas cantidatas'

    limit = kargs['limit'] if 'limit' in kargs else 5
    
    query = f'Usuario: {kargs["quer"]}.' if 'query' in kargs else '¿Me recomendarias una pelicula?'

    return f"""
Eres un servicio que recomienda películas a sus usuarios. Además te comunicas con tus usuarios en lenguaje español.

Quiero que recomiendes una película a un usuario basándote en información personal y 
registros históricos de películas vistas.

Perfil del usuario: {user_profile}.

Los registros históricos incluyen el nombre de la película, el tipo y cuántos puntos 
obtuvo de 5. Cuanto mayor es la puntuación, más le gusta la película. Te animamos a 
aprender su preferencia de películas de las películas que ha visto. Aquí hay algunos 
ejemplos:

{user_history}.

Aquí hay una lista de películas que probablemente le gustarán: {candidates}.

Por favor, selecciona las {limit} mejores películas de la lista que es más probable que le gusten.
La primera la película con mayor rating es la más cercana a los gustos del usuario. Por favor, selecciona las 4 películas
restantes. Solo muestra el nombre de la película y si identificador entre paréntesis.

En caso de no tener ninguna información para recomendar elige entre las películas más taquilleras. Siempre responde en lenguaje español.

El formato de la respuesta siempre debe especificarse con el siguiente formato:

Comienza lista de recommendaciones:
Número. Título(Año de estreno, Calificación numérica entre los valores 1 y 5): Descripción.
Fin de lista de recommendaciones:

El título, año de estreno, calificación y descripción debe especificarse en lenguaje natural y no deben estar entre comillas.

{query}
"""

In [11]:
class OllamaAssitant:
    def __init__(self, model, promp_build_fn):
        self.__api_client     = OllamaApiClient(model=model)
        self.__promp_build_fn = promp_build_fn

    def recommend(self, **kargs):
        promp = self.__promp_build_fn(**kargs)
        return self.__api_client.query(promp)

## Model Inference

In [12]:
assistant = OllamaAssitant('movie_recommender', promp_build_fn)

In [13]:
result = assistant.recommend(
 user_profile = 'Tiene 24 años y trabaja como técnico'
)

In [16]:
output = MovieRecommenderOutputParser().parse(result.response)

In [17]:
print(output)

[
    {
        'position': 1,
        'title': 'El laberinto del fauno',
        'release': 2006,
        'rating': 4.5,
        'description': 'Un drama fantástico que combina elementos de terror y aventura para crear una historia 
impactante y emocionante. dirigida por guillermo del toro y protagonizada por tim roth, michael lloreda y ivana 
baquero.'
    },
    {
        'position': 2,
        'title': 'La la land',
        'release': 2016,
        'rating': 4.2,
        'description': 'Una comedia romántica que combina canciones y baile para contar la historia de dos jóvenes 
que luchan por alcanzar sus sueños en los ángeles. dirigida por damien chazelle y protagonizada por ryan gosling y 
emma stone.'
    },
    {
        'position': 3,
        'title': 'La habitación del tigr',
        'release': 2017,
        'rating': 4.8,
        'description': 'Un thriller psicológico que sigue a una joven que investiga la desaparición de su hermano 
en un pueblo remoto. dirigida por ari aster y protagonizada por florence pugh y jack reynor.'
    },
    {
        'position': 4,
        'title': 'Roma',
        'release': 2018,
        'rating': 4.5,
        'description': 'Una película dramática que cuenta la historia de una mujer que trabaja como niñera en 
méxico durante la década de 1970. dirigida por alfonso cuarón y protagonizada por yalitza aparicio.'
    },
    {
        'position': 5,
        'title': 'Hereditary',
        'release': 2018,
        'rating': 4.3,
        'description': 'Un thriller psicológico que sigue a una familia después de la muerte de su madre, 
descubriendo secretos oscuros y malevolentes. dirigida por ari aster y protagonizada por toni collette, alex wolff 
y milly shapiro.'
    }
]